# Introduction

TBD

In [ ]:
import pymc3 as pm
import theano.tensor as tt
import numpy as np

In [ ]:
# We start with parental weights of length 4, one for each feature.
parental_weights = np.random.normal(loc=10, scale=3, size=(4,))
parental_weights

In [ ]:
# We'll now generate new weights based on location=parental_weights,
# and scale=1

child_weights = np.random.normal(loc=parental_weights, scale=1, size=(2, 4))
child_weights

These are the true weights of the system.

We are now going to attempt to learn them in a Bayesian fashion.

In [ ]:
n_samps = 100
n_tasks = 2
n_weights = 4
data = np.random.normal(loc=3, scale=4, size=(n_tasks, n_samps, n_weights))

# Now, on the last 20 samples on the 2nd task, set everything to zeros
# to indicate that it has nothing in there.
data[80:, 1] = 0

In [ ]:
data.shape, child_weights.shape

In [ ]:
y = np.einsum("ijk, ik -> ij", data, child_weights)

In [ ]:
y.shape

We are now going to write a hierarchical linear regression model that handles this particular case of imbalanced number of samples.

If we are able to recover back the original weights, then zero-padding could be a very powerful technique to deal with multiple learning tasks that also have non-equal numbers of samples that also have non-overlapping sample indices.

In [ ]:
data.shape

In [ ]:
child_weights.shape

In [ ]:
tt.batched_dot(data, child_weights)

In [ ]:
data.shape

In [ ]:
y.shape

In [ ]:
with pm.Model() as hierarchical_linear_model:
    w_parent = pm.Normal("w_parent", mu=0, sd=1, shape=(4,))

    # Broadcasting will give us 4 child weights drawn from w_parent,
    # I think.
    w_child = pm.Normal("w_child", mu=w_parent, sd=1, shape=(2, 4))

    sd = pm.HalfCauchy("sd", beta=10)

    # mu = pm.Deterministic("mu", np.einsum('ijk, kj -> ij', data, w_child))
    mu = pm.Deterministic("mu", tt.batched_dot(data, w_child))
    like = pm.Normal("like", mu=mu, sd=sd, observed=y)

In [ ]:
with hierarchical_linear_model:
    # trace = pm.sample(2000, cores=1)
    approx = pm.fit(100000)
    trace = approx.sample(2000)

In [ ]:
trace["w_child"]

In [ ]:
trace["w_parent"].mean(axis=0)

In [ ]:
parental_weights

We're close!

In [ ]:
trace["w_child"].mean(axis=0)

In [ ]:
trace["w_child"].std(axis=0)

In [ ]:
child_weights

OK! I think that this works, just that something is not right with NUTS because of gradient issues (we get zeros on diagonal of mass matrix). I'm going to show this experiment to the PyMC devs to see what I might be doing wrong.

# Recap

Just to recap what we've done here.

We have two learning tasks that involve the same _kind_ of input data, but don't have exactly aligned samples. In the first learning task, we have 100 iid samples; in the 2nd learning task we have 80 iid samples. In our data matrix, the 2nd task's 80 iid samples are not necessarily aligned with the 100 iid samples from the 1st task. One other assumption we have baked into this model is that the weights, while given a set for each task, are shared from a parental prior, hence there is parameter sharing amongst the learning tasks, though not in our usual "classical" sense.

By appending zero-padding, we should be able to generalize this to multi-task neural network learning with non-overlapping samples. [Thomas Wiecki](https://twiecki.io/blog/2018/08/13/hierarchical_bayesian_neural_network/) has a great blog post on how to do it, though he didn't deal with the "number of samples" issue, which I tried to add here.